In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-29 15:30:30.469176: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 15:30:30.492460: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 15:30:30.492483: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 15:30:30.492508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 15:30:30.497850: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-29 15:30:31.926471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:30:31.948723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:30:31.951100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-29 15:30:32.277228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:30:32.279820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:30:32.282647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.nasnet.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.NASNetMobile(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [13]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: stem_conv1, False
Layer: stem_bn1, False
Layer: activation, False
Layer: reduction_conv_1_stem_1, False
Layer: reduction_bn_1_stem_1, False
Layer: activation_1, False
Layer: activation_3, False
Layer: separable_conv_1_pad_reduction_left1_stem_1, False
Layer: separable_conv_1_pad_reduction_right1_stem_1, False
Layer: separable_conv_1_reduction_left1_stem_1, False
Layer: separable_conv_1_reduction_right1_stem_1, False
Layer: separable_conv_1_bn_reduction_left1_stem_1, False
Layer: separable_conv_1_bn_reduction_right1_stem_1, False
Layer: activation_2, False
Layer: activation_4, False
Layer: separable_conv_2_reduction_left1_stem_1, False
Layer: separable_conv_2_reduction_right1_stem_1, False
Layer: activation_5, False
Layer: separable_conv_2_bn_reduction_left1_stem_1, False
Layer: separable_conv_2_bn_reduction_right1_stem_1, False
Layer: separable_conv_1_pad_reduction_right2_stem_1, False
Layer: activation_7, False
Layer: reduction_add_1_stem_1, False
Layer: s

In [14]:
len(base_model.layers)

769

In [15]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [16]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [17]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
#x = aug_layer(inputs)
#x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(inputs, training = False)
x = layers.GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'NasnetMobile_89.23')

model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [18]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [19]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [20]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [21]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 15, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/15


2024-07-29 15:32:41.760812: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-29 15:32:41.820914: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-29 15:32:43.036416: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x762e5c003470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-29 15:32:43.036441: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-29 15:32:43.041727: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-29 15:32:43.100937: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 11.8785 - accuracy: 0.6374
Epoch 1: val_loss improved from inf to 9.43202, saving model to Trained_Models/Nasnet
74/74 [==============================] - 38s 410ms/step - loss: 11.8785 - accuracy: 0.6374 - val_loss: 9.4320 - val_accuracy: 0.8764 - lr: 0.0010
Epoch 2/15
74/74 [==============================] - ETA: 0s - loss: 7.3012 - accuracy: 0.7798
Epoch 2: val_loss improved from 9.43202 to 5.57450, saving model to Trained_Models/Nasnet
74/74 [==============================] - 30s 401ms/step - loss: 7.3012 - accuracy: 0.7798 - val_loss: 5.5745 - val_accuracy: 0.9318 - lr: 0.0010
Epoch 3/15
74/74 [==============================] - ETA: 0s - loss: 4.3811 - accuracy: 0.8098
Epoch 3: val_loss improved from 5.57450 to 3.41565, saving model to Trained_Models/Nasnet
74/74 [==============================] - 29s 391ms/step - loss: 4.3811 - accuracy: 0.8098 - val_loss: 3.4157 - val_accuracy: 0.9624 - lr: 0.0010
Epoch 4/15
73/74 [========

In [22]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 43ms/step - loss: 0.6405 - accuracy: 0.8663


[0.6404761075973511, 0.8662722110748291]

In [23]:
base_model.trainable = True

In [24]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [25]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [26]:
start_epoch = 15
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [27]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 16/25
73/74 [============================>.] - ETA: 0s - loss: 0.7677 - accuracy: 0.8390
Epoch 16: val_loss did not improve from 0.53758
74/74 [==============================] - 13s 73ms/step - loss: 0.7670 - accuracy: 0.8391 - val_loss: 0.6958 - val_accuracy: 0.8892 - lr: 0.0010
Epoch 17/25
74/74 [==============================] - ETA: 0s - loss: 0.7121 - accuracy: 0.8695
Epoch 17: val_loss did not improve from 0.53758
74/74 [==============================] - 4s 52ms/step - loss: 0.7121 - accuracy: 0.8695 - val_loss: 0.5452 - val_accuracy: 0.9604 - lr: 0.0010
Epoch 18/25
73/74 [============================>.] - ETA: 0s - loss: 0.6763 - accuracy: 0.8720
Epoch 18: val_loss did not improve from 0.53758
74/74 [==============================] - 4s 52ms/step - loss: 0.6770 - accuracy: 0.8712 - val_loss: 0.6540 - val_accuracy: 0.9337 - lr: 0.0010
Epoch 19/25
74/74 [==============================] - ETA: 0s - loss: 0.6212 - accuracy: 0.8818
Epoch 19: val_loss improved from 0.53758 to 0.

In [28]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 35ms/step - loss: 0.5579 - accuracy: 0.8615


[0.5578933358192444, 0.8615384697914124]

In [29]:
start_epoch = 25
for layer in base_model.layers[-75:]:
    layer.trainable = True

In [30]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [31]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 26/35
73/74 [============================>.] - ETA: 0s - loss: 0.4235 - accuracy: 0.9114
Epoch 26: val_loss did not improve from 0.32211
74/74 [==============================] - 17s 81ms/step - loss: 0.4242 - accuracy: 0.9111 - val_loss: 0.7964 - val_accuracy: 0.7685 - lr: 3.0000e-04
Epoch 27/35
74/74 [==============================] - ETA: 0s - loss: 0.3826 - accuracy: 0.9183
Epoch 27: val_loss improved from 0.32211 to 0.30892, saving model to Trained_Models/Nasnet
74/74 [==============================] - 31s 426ms/step - loss: 0.3826 - accuracy: 0.9183 - val_loss: 0.3089 - val_accuracy: 0.9614 - lr: 3.0000e-04
Epoch 28/35
74/74 [==============================] - ETA: 0s - loss: 0.3423 - accuracy: 0.9339
Epoch 28: val_loss did not improve from 0.30892
74/74 [==============================] - 5s 58ms/step - loss: 0.3423 - accuracy: 0.9339 - val_loss: 0.3647 - val_accuracy: 0.9268 - lr: 3.0000e-04
Epoch 29/35
73/74 [============================>.] - ETA: 0s - loss: 0.2981 - accura

In [32]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 36ms/step - loss: 0.4785 - accuracy: 0.8899


[0.4785414934158325, 0.8899408578872681]

In [33]:
start_epoch = 35
base_model.trainable = True
for layer in base_model.layers[:-125]:
    layer.trainable = False

In [34]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000075), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [35]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 36/45
73/74 [============================>.] - ETA: 0s - loss: 0.1767 - accuracy: 0.9816
Epoch 36: val_loss did not improve from 0.20760
74/74 [==============================] - 24s 129ms/step - loss: 0.1771 - accuracy: 0.9814 - val_loss: 0.3641 - val_accuracy: 0.9110 - lr: 7.5000e-05
Epoch 37/45
74/74 [==============================] - ETA: 0s - loss: 0.1477 - accuracy: 0.9890
Epoch 37: val_loss did not improve from 0.20760
74/74 [==============================] - 5s 66ms/step - loss: 0.1477 - accuracy: 0.9890 - val_loss: 0.2148 - val_accuracy: 0.9644 - lr: 7.5000e-05
Epoch 38/45
73/74 [============================>.] - ETA: 0s - loss: 0.1407 - accuracy: 0.9863
Epoch 38: val_loss improved from 0.20760 to 0.20575, saving model to Trained_Models/Nasnet
74/74 [==============================] - 31s 424ms/step - loss: 0.1404 - accuracy: 0.9864 - val_loss: 0.2057 - val_accuracy: 0.9664 - lr: 7.5000e-05
Epoch 39/45
74/74 [==============================] - ETA: 0s - loss: 0.1672 - accur

In [36]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 35ms/step - loss: 0.4816 - accuracy: 0.8828


[0.48157748579978943, 0.8828402161598206]

In [37]:
start_epoch = 45
for layer in base_model.layers[-300:]:
    layer.trainable = True

In [38]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [39]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 46/55
74/74 [==============================] - ETA: 0s - loss: 0.1190 - accuracy: 0.9856
Epoch 46: val_loss did not improve from 0.16067
74/74 [==============================] - 36s 107ms/step - loss: 0.1190 - accuracy: 0.9856 - val_loss: 0.2369 - val_accuracy: 0.9585 - lr: 2.5000e-05
Epoch 47/55
74/74 [==============================] - ETA: 0s - loss: 0.1114 - accuracy: 0.9894
Epoch 47: val_loss did not improve from 0.16067
74/74 [==============================] - 6s 83ms/step - loss: 0.1114 - accuracy: 0.9894 - val_loss: 0.4673 - val_accuracy: 0.8724 - lr: 2.5000e-05
Epoch 48/55
74/74 [==============================] - ETA: 0s - loss: 0.1195 - accuracy: 0.9886
Epoch 48: val_loss did not improve from 0.16067
74/74 [==============================] - 6s 83ms/step - loss: 0.1195 - accuracy: 0.9886 - val_loss: 0.2321 - val_accuracy: 0.9525 - lr: 2.5000e-05
Epoch 49/55
74/74 [==============================] - ETA: 0s - loss: 0.0976 - accuracy: 0.9936
Epoch 49: val_loss did not improv

In [40]:
start_epoch = 55
for layer in base_model.layers[-700:]:
    layer.trainable = True

In [41]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [42]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 56/65
74/74 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9814
Epoch 56: val_loss did not improve from 0.16067
74/74 [==============================] - 55s 148ms/step - loss: 0.1236 - accuracy: 0.9814 - val_loss: 0.3335 - val_accuracy: 0.9308 - lr: 2.5000e-05
Epoch 57/65
74/74 [==============================] - ETA: 0s - loss: 0.1372 - accuracy: 0.9809
Epoch 57: val_loss did not improve from 0.16067
74/74 [==============================] - 8s 107ms/step - loss: 0.1372 - accuracy: 0.9809 - val_loss: 0.1625 - val_accuracy: 0.9763 - lr: 2.5000e-05
Epoch 58/65
74/74 [==============================] - ETA: 0s - loss: 0.1013 - accuracy: 0.9903
Epoch 58: val_loss did not improve from 0.16067
74/74 [==============================] - 8s 108ms/step - loss: 0.1013 - accuracy: 0.9903 - val_loss: 0.2233 - val_accuracy: 0.9594 - lr: 2.5000e-05
Epoch 59/65
74/74 [==============================] - ETA: 0s - loss: 0.0955 - accuracy: 0.9911
Epoch 59: val_loss did not impr

In [43]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 35ms/step - loss: 0.4736 - accuracy: 0.8923


[0.47362667322158813, 0.892307698726654]

In [38]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet")


OSError: No file or directory found at /home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet

In [ ]:
resnet.evaluate(val_datagen)